## Import Library
Import all libraries required to run the functional genetic algorithm

In [36]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt

## City & Fitness Class

The city class functions to make it easier to define cities coordinate and calculate the distance between two cities. while the fitness class functions to determine the route taken and calculate the total distance and fitness value

In [37]:
class City:
    def __init__(self,name, x, y):
        self.x = x
        self.y = y
        self.name = name

    def distance(self, city):
        xDis = abs(self.x - city.x)
        yDis = abs(self.y - city.y)
        distance = np.sqrt((abs(self.x - city.x) ** 2) + (abs(self.y - city.y)** 2))
        return distance

    def __repr__(self):
        return "("+str(self.name)+")"

In [38]:
class Fitness:
    def __init__(self, route):
        self.route = route
        self.distance = 0
        self.fitness = 0.0

    #menghitung jarak total setiap rute
    def routeDistance(self):
        if self.distance == 0:
            #simpan jarak sementar
            pathDistance = 0
            for i in range(0, len(self.route)):
                #kota awal
                fromCity = self.route[i]
                toCity = None
                #mengecek jika kota selanjutnya masih ada
                if i + 1 < len(self.route):
                    toCity = self.route[i + 1]
                else:
                    #balik ke titik awal
                    toCity = self.route[0]
                pathDistance += fromCity.distance(toCity)
            self.distance = pathDistance
        return self.distance

    #menghitung nilai fitness setiap rute
    def routeFitness(self):
        if self.fitness == 0:
            self.fitness = 1 / float(self.routeDistance())
        return self.fitness

    #menampilakan namaKota
    def printCity(self):
        namaKota =[]
        for i in range(0,len(self.route)):
            namaKota.append(self.route[i].name)
        return "("+str(namaKota)+ ")"

## Genetic Algorithm Functions

Create several functions to perform each step in the genetic algorithm, such as:
- Create Route
- Initial Population
- Rank Routes
- Selection
- Check Tabulist
- Crossover
- Mutate
- Update Generation

### Create Route

In [39]:
def createRoute(cityList):
    #mengacak urutan kota
    route = random.sample(cityList, len(cityList))
    return route

### Intialization the Population

In [40]:
def initialPopulation(popSize, cityList):
    population = []
    # membuat populasi sesuai jumlah popSize
    for i in range(0, popSize):
      route = createRoute(cityList)
      while route in population:
        route = createRoute(cityList)
      population.append(createRoute(cityList))
    return population

### Rank Routes

In [41]:
def rankRoutes(population):
    fitnessResults = []
    rankRoute = {}

    #menghitung nilai fitnes dari setiap individu
    for i in range(0, len(population)):
        fitnessResults.append(Fitness(population[i]).routeFitness())
    #mengurutkan nilai fitnes
    sortedFitness = sorted(fitnessResults, reverse=True)

    #mengurutkan individu berdasarkan nilai fitnes
    for i in range(0, len(sortedFitness)):
        for j in range(0, len(population)):
            if sortedFitness[i] == Fitness(population[j]).routeFitness():
                rankRoute[i] = population[j]

    return rankRoute

### Selection

In [42]:
def selection(population):
    selection = []
    #mengambil 2 individu terbaik berdasarkan nilai fitness
    for i in range(0,2):
        selection.append(population[i])
        print("seleksi ke-" + str(i+1) + " " + str(Fitness(selection[i]).printCity()) + "\n" +
              " Jarak = " + str(Fitness(selection[i]).routeDistance()) + "\n" + " Fitness = " + str(
            Fitness(selection[i]).routeFitness()))
    return selection

### Tabulist

In [43]:
def tabulist(selectionResults,tabulist):
    isTabulist1 = False
    isTabulist2 = False
    isTabulist = False

    #cek tabulist
    for j in range(0,len(tabulist)):
        if selectionResults[0] == tabulist[j]:
            isTabulist1 = True
        if selectionResults[1] == tabulist[j]:
            isTabulist2 = True

    if isTabulist1 == False:
        tabulist.append(selectionResults[0])
    if isTabulist2 == False:
        tabulist.append(selectionResults[1])

    if isTabulist1 == True and isTabulist2 == True:
        isTabulist = True

    return isTabulist,tabulist

### Crossover

In [44]:
def crossover(parent1, parent2,tab):
    childP1 = []
    childP2 = []
    child = []
    # mengacak nomor gen yang akan dilakukan crossover
    # jika tabulis true maka hasil crossover sama dengan parent1(individu terbaik)
    if tab == True:
        geneA = 0
        geneB = len(parent1)-1
    else:
        geneA = random.randint(1, len(parent1)-1)
        geneB = random.randint(1, len(parent1)-1)
        # melakukan perulangan agar gen A dan gen B tidak bernilai sama
        while geneA == geneB:
            geneB = random.randint(1, len(parent1)-1)

    startGene = min(geneA, geneB)
    endGene = max(geneA, geneB)
    print("stargene = "+str(startGene+1)+", endGene = "+str(endGene+1))

    for i in range(startGene, endGene+1):
        childP1.append(parent1[i])

    for item in parent2:
        if item not in childP1:
            childP2.append(item)

    #hasil crossover
    idxChild1 = 0
    idxChild2 = 0
    for i in range(len(parent1)):
      if i >= startGene and i <= endGene:
        child.append(childP1[idxChild1])
        idxChild1 += 1
      else:
        child.append(childP2[idxChild2])
        idxChild2 += 1

    return child

### Mutate

In [45]:
def mutate(individual):
    #mengacak nomor gen yang akan di tukar
    swapped = random.randint(1, len(individual)-1)
    swapWith = random.randint(1, len(individual)-1)
    ## melakukan perulangan agar tidak muncul nilai sama
    while swapped == swapWith:
        swapWith = random.randint(1, len(individual)-1)

    city1 = individual[swapped]
    city2 = individual[swapWith]

    print("swapped = "+str(swapped+1) + " , swap with = " + str(swapWith+1))

    #hasil mutasi
    individual[swapped] = city2
    individual[swapWith] = city1
    return individual

### Update Generation

In [46]:
def updateGeneration(population,popsize,mutate):
    generasibaru =[]
    # menggabungkan populasi dengan hasil mutasi untuk dijadikan generasi baru
    for i in range(0,popsize):
        generasibaru.append(population[i])
    # mengganti individu dengan fitness terkecil(index terakhir) dengan hasil mutasi jika hasil mutasi lebih besar
    if Fitness(mutate).routeFitness() >= Fitness(generasibaru[-1]).routeFitness():
      generasibaru[popsize-1] =mutate
    for i in range(0,popsize):
        print("Generasi baru ke-" + str(i+1) + " " + str(Fitness(generasibaru[i]).printCity()) +
              " Jarak = " + str(Fitness(generasibaru[i]).routeDistance()) + " Fitness = " + str(
            Fitness(generasibaru[i]).routeFitness()))

    return generasibaru

### Implement The Genetica Algorithm

In [47]:
def geneticAlgorithm(population, popSize, generations):
    tabulistResult = []
    finalPopulation = []
    populasi = initialPopulation(popSize, population)
    rankPopulasi = rankRoutes(populasi)
    pop = []

    # Mengurutkan setiap populasi berdasarkan ranking
    for i in range(0, popSize):
        pop.append(rankPopulasi[i])

    bestDistance = 9999999999999
    progress =[]

    populasi = initialPopulation(popSize, population)
    rankPopulasi = rankRoutes(populasi)
    pop = []

    # Mengurutkan setiap populasi berdasarkan ranking
    for i in range(0, popSize):
        pop.append(rankPopulasi[i])
    print("=========================== GENERASI KE " + str(1) + "===============================================================>")
    for i in range(0,len(pop)):
        print("populasi ke-" + str(i+1) + " " + str(Fitness(pop[i]).printCity()) +
              " Jarak = " + str(Fitness(pop[i]).routeDistance()) + " Fitness = " + str(
            Fitness(pop[i]).routeFitness()))
    print("==========================================================================================>")

    bestDistance = 9999999999999
    progress =[]
    for i in range(0, generations):
        nextGeneration = []
        selectionResults = selection(pop)
        tab, tabulistResult = tabulist(selectionResults, tabulistResult)

        crossoverResult = crossover(selectionResults[0], selectionResults[1], tab)
        if tab == False:
            print("Tabulist False")
            print("==========================================================================================>")
            print("hasil crossover = " + str(crossoverResult))
            print("==========================================================================================>")
        elif tab == True:
            print("Tabulist true")
            print("==========================================================================================>")
            print("seleksi 1 = " + str(crossoverResult))
            print("==========================================================================================>")
        print("TABULIST : ", tabulistResult, "\n")
        children = mutate(crossoverResult)
        print("hasil mutasi = " + str(children) + " Jarak = " + str(Fitness(children).routeDistance()) + " Fitness = " + str(
            Fitness(children).routeFitness()))
        print("==========================================================================================>")
        nextGeneration = updateGeneration(pop, popSize, children)
        finalPopulation = nextGeneration
        poptemp = nextGeneration

        # Mengurutkan generasi baru dengan ranking
        poptemp = rankRoutes(poptemp)
        pop = []

        for j in range(0, popSize):
            pop.append(poptemp[j])

        finalPopulation = pop
        print("\n====================================== GENERASI KE " + str(
            i + 2) + "======================================>")
        for n in range(0, len(pop)):
            print("populasi ke-" + str(n+1) + " " + str(Fitness(pop[n]).printCity()) +
                " Jarak = " + str(Fitness(pop[n]).routeDistance()) + " Fitness = " + str(
                Fitness(pop[n]).routeFitness()))
        print("=========================================================================================>")

        tempbestroute = Fitness(pop[0]).routeDistance()

        # Untuk membuat plot berdasarkan rute terbaiknya
        progress.append(Fitness(pop[0]).routeDistance())

        if (bestDistance > tempbestroute):
            bestIndividu = Fitness(pop[0]).printCity()
            bestDistance = Fitness(pop[0]).routeDistance()
            bestFitness = Fitness(pop[0]).routeFitness()

    print("BEST INDIVIDU = " + str(bestIndividu))
    print("BEST DISTANCE = " + str(bestDistance))
    print("BEST FITNESS = " + str(bestFitness))

    plt.figure(0)
    plt.plot(progress)
    plt.ylabel('Distance')
    plt.xlabel('Generation')
    plt.title('Genetica Algorithm Result')
    plt.savefig('algen_result.png')

    return finalPopulation

## Ant Colony Optimization Function

In [48]:
def antColonyOptimization(city, iteration, nAnts, rho, alpha, beta, initialPheromne, routes):
  cityList = []
  for i in range(0, len(city)):
    cityList.append(City(name = city.iloc[i,0],x=city.iloc[i][1],y=city.iloc[i][2]))

  distances = np.zeros((len(cityList), len(cityList)))
  visibility = np.zeros((len(cityList), len(cityList)))
  # Inisialization Pheromne
  pheromne = initialPheromne * np.ones((len(cityList), len(cityList)))

  for row in range(len(cityList)):
    for col in range (len(cityList)):
      distance = cityList[row].distance(cityList[col])
      distances[row, col] = distance
      visibility[row, col] = 1/distance  if distance != 0 else 0

  totalDistance = np.zeros((len(routes), 1)) # initiate total distance
  print(f"Genetica Algorithm Result: ")
  for i in range(len(routes)):
    routeStr = ""
    distance = 0
    for j in range(len(routes[i])-1):
      distance += routes[i][j].distance(routes[i][j+1])
      routeStr += " "+str(routes[i][j].name)
    routeStr += " "+str(routes[i][-1].name)
    print(f"Semut {i+1}: [{routeStr}] Distance: {Fitness(routes[i]).routeDistance()}")
    totalDistance[i] = distance

  # Update pheromne with genetica algorithm result
  pheromne = (1 - rho) * pheromne # Evaporation
  for i in range(nAnts):
    delta = 1 / totalDistance[i][0] # Delta Pheromne
    for j in range(len(cityList)):
      pheromne[int(routes[i][j].name) - 1, int(routes[i][j+1].name) - 1] += delta
      pheromne[int(routes[i][j + 1].name) - 1, int(routes[i][j].name) - 1] += delta

  print("\nInitial Pheromne")
  print("--------------------------------")
  print("Initail City | Destination City | Pheromne")
  for row in range(len(cityList)):
    for col in range (row+1,len(cityList)):
      print(f"{row + 1}             | {col + 1}               | {pheromne[row, col]:.4f}")
  print("------------------------------------------\n")

  bestRoute = None
  bestDistance = float('inf')
  bestDistances = []

  # ACO Iteration
  for idx in range(iteration):
    print(f"=========================== Iteration {idx+1} ============================")
    # Initialization Routes
    routes = np.ones((nAnts, len(cityList)+1), dtype=int)

    antAndDistanceStr = ""
    # Initialization the first City index
    initialCitiesIdx = np.random.permutation(len(cityList))
    totalDistance = np.zeros((nAnts, 1)) # initiate total distance
    for i in range(nAnts):
      distance = 0
      # Assign first city to each ants
      routes[i, 0] = initialCitiesIdx[i] + 1
      # Copy the visibility
      visibilityTemp = np.array(visibility)

      for j in range(len(cityList)-1):
        currentLocation = int(routes[i, j] -1)
        # Set City Visibility to zero
        visibilityTemp[:, currentLocation] = 0

        pFeature = np.power(pheromne[currentLocation, :], beta)
        vFeature = np.power(visibilityTemp[currentLocation, :], alpha)
        features = np.multiply(pFeature, vFeature)
        total = np.sum(features)
        probabilities = features/total

        print(f"Semut {i + 1}: {routes[i, :]}")
        print("---------------------")
        print("Kota  | Probabilitas |")
        for k in range(len(cityList)):
          print(f"{k + 1}     | {probabilities[k]:.4f}       |")
        print("---------------------")

        # Choose next city with highest probability
        nextCityIdx = np.argmax(probabilities)
        routes[i, j+1] = nextCityIdx + 1 # Add next city to route

        distance += distances[int(routes[i, j]) - 1, int(routes[i, j+1]) - 1]


      routes[i, -1] = routes[i, 0] # Back to first City
      print(f"Semut {i + 1}: {routes[i, :]}")

      # Calculate last city to first city
      distance += distances[int(routes[i, -2]) - 1, int(routes[i, -1]) - 1]
      totalDistance[i] = distance
      antAndDistanceStr += f"Semut {i+1}: {'-'.join(map(str, map(int, routes[i, :])))} | Jarak = {totalDistance[i, 0]:.4f}\n"

      print("\n====================\n")

    print(f"Iteration {idx} Result: ")
    print(antAndDistanceStr)


    # Search the best routes
    distanceMinIdx = np.argmin(totalDistance)
    distanceMin = totalDistance[distanceMinIdx]
    if distanceMin < bestDistance:
      bestDistance = distanceMin
      bestRoute = routes[distanceMinIdx, :]

    bestDistances.append(bestDistance)

    # Update pheromne
    pheromne = (1 - rho) * pheromne # Evaporation
    for i in range(nAnts):
      delta = 1 / totalDistance[i][0] # Delta Pheromne
      for j in range(len(cityList)):
        pheromne[int(routes[i, j]) - 1, int(routes[i, j+1]) - 1] += delta
        pheromne[int(routes[i, j + 1]) - 1, int(routes[i, j]) - 1] += delta

    print("Update Pheromne")
    print("--------------------------------")
    print("Initail City | Destination City | New Pheromne")
    for i in range(len(cityList)):
      for j in range(i+1, len(cityList)):
        pheromneValue = pheromne[i, j]
        print(f"{i + 1}             | {j + 1}               | {pheromneValue:.4f}")
    print("------------------------------------------\n")


  print(f"The best routes: {'-'.join(map(str, map(int, bestRoute)))} | Total Distance = {bestDistance[0]:.4f}")

  # Ploting ACO Result
  plt.figure(1)
  plt.plot(range(1, iteration + 1), bestDistances)
  plt.xlabel('Iteration')
  plt.ylabel('Distance')
  plt.title('Ant Colony Optimization Result')
  plt.savefig('aco_result.png')

## Main

In [50]:
%%capture cap
DATASET_PATH = "t5.csv"

# Algen Parameters
POPULATION_SIZE = 5
GENERATION = 2

# ACO Parameters
Iteration = 3
nAnts = 5
rho = 0.5
alpha = 1
beta = 1
InitialPheromne = 10


city = pd.read_csv(DATASET_PATH, header=None , sep=' ')
cityList = []
for i in range(0,len(city)):
    cityList.append(City(name = city.iloc[i,0],x=city.iloc[i][1],y=city.iloc[i][2]))

print("\n====================================== Genetica Algorithm ======================================\n")
print(f"Population Size: {str(POPULATION_SIZE)}")
print(f"Generation: {str(GENERATION)}")
print()

result = geneticAlgorithm(population=cityList, popSize=POPULATION_SIZE, generations=GENERATION)

newPop = []
for r in result:
  firstCity = r[0]
  newRoute = []
  for r2 in r:
    newRoute.append(r2)
  newRoute.append(firstCity)
  newPop.append(newRoute)



print("\n\n====================================== Ant Colony Optimization ======================================\n")
print(f"Iteration: {str(Iteration)}")
print(f"Ants: {str(nAnts)}")
print(f"Rho: {str(rho)}")
print(f"Alpha: {str(alpha)}")
print(f"Beta: {str(beta)}")
print(f"Initial Pheromne : {str(InitialPheromne)}")
print()

antColonyOptimization(city, Iteration, nAnts, rho, alpha, beta, InitialPheromne, newPop)

with open('genetica_and_aco_result.txt', 'w') as f:
    f.write(cap.stdout)